In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/agents-intensive-capstone-project/Hackathon dataset.txt


# Tahoe Trip Planner — Agents Intensive - Capstone Project

This notebook implements a **multi-agent Tahoe Ski Trip Planner** using the **Google Agent Developer Kit (ADK)** and **Gemini 2.5 Flash**.

We focus on:
- **Free-form user requests** (“cheap 2-day beginner trip from San Jose next weekend”)
- **Structured requirement parsing**
- **Web research** via `google_search`
- **Tool-augmented context lookup** (static resort/lodging tables +  weather + search)
- **Itinerary generation + safety reflection**
- A **supervisor agent** that orchestrates all of the above

Design goals:
- Our design goal is to build Sierra Summit AI — an intelligent multi-agent Tahoe ski trip planner that transforms a simple user request into a complete itinerary with real-time insights, safety guidance, and structured reasoning. The system is crafted to fully meet the Agents Intensive Capstone rubric by showcasing multi-agent orchestration, tool integrations, transparent reasoning traces, and a lightweight evaluation framework.


### Install & Imports

In [2]:
!pip install google-adk google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 6.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich

In [4]:
import os
import google.generativeai as genai

In [5]:
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [6]:
from google.adk.agents import Agent
from google.adk.tools import AgentTool
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search

In [17]:
# ===========================
# Observability: Logger Setup
# ===========================
import logging
import json
import time


logger = logging.getLogger("tahoe_agent")
logger.setLevel(logging.INFO)

# Add handler if not already added (prevents duplicates when rerunning cells)
if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter("%(asctime)s | %(levelname)s | %(message)s")
    handler.setFormatter(formatter)
    logger.addHandler(handler)

logger.info("Tahoe agent logging initialized.")


2025-11-29 21:11:18,693 | INFO | Tahoe agent logging initialized.
INFO:tahoe_agent:Tahoe agent logging initialized.


### Static Tahoe Data (Resorts & Lodging)

In [8]:
# Simple static tables for Tahoe resorts & lodging.

RESORTS = [
    {
        "name": "Northstar California",
        "area": "North Lake Tahoe",
        "skill": "beginner_friendly",
        "approx_budget": "mid",
        "vibe": "family, mellow, scenic",
    },
    {
        "name": "Heavenly",
        "area": "South Lake Tahoe",
        "skill": "mixed",
        "approx_budget": "mid_high",
        "vibe": "lake views, mixed crowds",
    },
    {
        "name": "Kirkwood",
        "area": "South Lake Tahoe",
        "skill": "advanced",
        "approx_budget": "mid",
        "vibe": "steep, more remote, serious skiers",
    },
]

LODGING = [
    {
        "name": "Northstar Village Condo",
        "area": "North Lake Tahoe",
        "type": "condo",
        "budget": "mid",
    },
    {
        "name": "Budget Inn South Lake",
        "area": "South Lake Tahoe",
        "type": "motel",
        "budget": "low",
    },
    {
        "name": "Heavenly Village Hotel",
        "area": "South Lake Tahoe",
        "type": "hotel",
        "budget": "mid_high",
    },
]


### Python Tools (Weather + Resort/Lodging Lookup)

In [9]:
# These are Python functions that ADK automatically wraps as tools.
from typing import Dict

def weather_tool(location: str) -> Dict:
    """
    Very simple placeholder 'weather tool'.
    """
    start = time.time()
    result = {
        "location": location,
        "summary": "Light snow expected, cold but skiable.",
        "snow_chance": "high",
        "road_risk": "moderate",
    }
    logger.info(
        "weather_tool called",
        extra={
            "tool": "weather_tool",
            "location": location,
            "duration_s": round(time.time() - start, 3),
        },
    )
    return result


def resort_lookup_tool(skill_level: str, budget_level: str) -> Dict:
    """
    Filter RESORTS to find a rough match based on skill + (approx) budget.
    This is intentionally simple and deterministic.
    """
    start = time.time()
    skill_level_norm = (skill_level or "").lower()
    budget_level_norm = (budget_level or "").lower()

    chosen = RESORTS[0]
    for r in RESORTS:
        # simple matching rules
        if "beginner" in skill_level_norm and r["skill"] == "beginner_friendly":
            chosen = r
            break
        if "intermediate" in skill_level_norm and r["skill"] in ("beginner_friendly", "mixed"):
            chosen = r
            break
        if "advanced" in skill_level_norm and r["skill"] == "advanced":
            chosen = r
            break

    logger.info(
        "resort_lookup_tool called",
        extra={
            "tool": "resort_lookup_tool",
            "skill_level": skill_level,
            "budget_level": budget_level,
            "chosen_resort": chosen["name"],
            "duration_s": round(time.time() - start, 3),
        },
    )
    return chosen


def lodging_lookup_tool(area: str, budget_level: str) -> Dict:
    """
    Filter LODGING by area + budget, with a fallback.
    """
    start = time.time()
    area_norm = (area or "").lower()
    budget_level_norm = (budget_level or "").lower()

    chosen = LODGING[0]

    # first pass: match both area + budget
    for l in LODGING:
        if area_norm.split()[0] in l["area"].lower() and budget_level_norm in l["budget"]:
            chosen = l
            break

    # second pass: match by area only, if first pass didn't change
    if chosen is LODGING[0]:
        for l in LODGING:
            if area_norm.split()[0] in l["area"].lower():
                chosen = l
                break

    logger.info(
        "lodging_lookup_tool called",
        extra={
            "tool": "lodging_lookup_tool",
            "area": area,
            "budget_level": budget_level,
            "chosen_lodging": chosen["name"],
            "duration_s": round(time.time() - start, 3),
        },
    )
    return chosen


### Agents

#### Agent 1: TripRequirementParserAgent

In [10]:
#Parses free-form user text into a structured trip_request.
trip_requirement_parser_agent = Agent(
    name="TripRequirementParserAgent",
    model="gemini-2.5-flash",
    instruction="""
You are a Tahoe ski trip requirement parser.

Your ONLY job:
- Read the user's free-form request.
- Extract the key trip requirements into a JSON object called trip_request.

trip_request MUST have this structure:
{
  "origin": str,                 // e.g. "San Jose"
  "trip_type": str,              // "day_trip" or "overnight"
  "num_days": int,               // typically 1 or 2
  "skill_level": str,            // "beginner", "intermediate", "advanced"
  "budget_level": str,           // "low", "mid", "high"
  "timing_flexibility": str,     // e.g. "next weekend", "any weekend next month"
  "preferences": str             // e.g. "scenic views", "family friendly", etc.
}

Rules:
- If the user does not say, make a reasonable guess and mark it clearly in the value
  (e.g. "unknown (assumed beginner)").
- Do NOT call any tools.
- Do NOT add extra commentary; ONLY return JSON.
""",
    output_key="trip_request",
)


#### Agent 2: TripSearchAgent (uses google_search)

In [11]:
#This agent is responsible for real-world web research about Tahoe road/weather advisories.
trip_search_agent = Agent(
    name="TripSearchAgent",
    model="gemini-2.5-flash",
    instruction="""
You are a Tahoe trip research assistant.

Your job:
- Use the google_search tool to look up current public information about Lake Tahoe
  winter road and weather advisories near the trip time.
- Focus on Caltrans chain controls, winter driving advisories, and major storm warnings.

Return a JSON object called road_info:

{
  "query_used": str,
  "summary": str,
  "top_insights": [str, ...]
}

Rules:
- You MUST call google_search at least once.
- summary should be 1–2 sentences, giving a concise overview of road / weather conditions.
- Keep top_insights to 2–3 short bullet-style sentences.
- Do NOT include any text outside the JSON.
""",
    tools=[google_search],
    output_key="road_info",
)


#### Agent 3: TripContextAgent

In [12]:
trip_context_agent = Agent(
    name="TripContextAgent",
    model="gemini-2.5-flash",
    instruction="""
You receive:
- trip_request
- road_info  (output of TripSearchAgent)

Your responsibilities:
1. Pick a resort using resort_lookup_tool(skill_level, budget_level).
2. Infer the Tahoe area ("North Lake Tahoe" or "South Lake Tahoe").
3. Pick lodging using lodging_lookup_tool(area, budget_level).
4. Get a simple weather estimate using weather_tool(area).
5. Use road_info.summary exactly as road_summary.

Return a JSON object called trip_context:
{
  "resort": {...},
  "lodging": {...},
  "weather": {...},
  "road_summary": str,
  "area": str
}

Rules:
- MUST call resort_lookup_tool, lodging_lookup_tool, and weather_tool.
- MUST NOT call google_search.
- No text outside JSON.
""",
    tools=[weather_tool, resort_lookup_tool, lodging_lookup_tool],
    output_key="trip_context",
)


#### Agent 4: TripItineraryAgent

In [13]:
trip_itinerary_agent = Agent(
    name="TripItineraryAgent",
    model="gemini-2.5-flash",
    instruction="""
You receive:
- trip_request: {trip_request}
- trip_context: {trip_context}
- road_info: {road_info}

Your job:
- Create a simple, friendly 1–2 day Tahoe ski trip itinerary.
- Use the chosen resort, lodging, weather, and road_summary from trip_context.
- ALSO incorporate 1 short insight from road_info.top_insights
  (e.g., chain controls, winter advisory, storm warnings).

Requirements:
- If road_info.top_insights is available, mention it once as a short note.
- Use morning/afternoon/evening suggestions.
- Output a short Markdown bullet list itinerary.
- Keep it concise and beginner-friendly.
""",
    output_key="itinerary",
)


#### Agent 5: TripSafetyAgent

In [14]:
trip_safety_agent = Agent(
    name="TripSafetyAgent",
    model="gemini-2.5-flash",
    instruction="""
You receive:
- trip_request: {trip_request}
- trip_context: {trip_context}
- itinerary: {itinerary}
- road_info: {road_info}

Your job:
- Identify safety issues for a beginner skier and Tahoe winter traveler.
- You MUST consider:
    • trip_context.weather
    • trip_context.road_summary
    • ANY insights in road_info.top_insights (e.g., chain controls, storm alerts)
- Suggest simple, practical mitigations.

Return a JSON object called safety_notes:

{
  "risks": [...],
  "mitigations": [...]
}

Rules:
- Do NOT include any text outside JSON.
- Keep risks and mitigations beginner-friendly.
""",
    output_key="safety_notes",
)


#### Agent 6: TripSupervisorAgent (Supervisor / Orchestrator)

In [15]:
#Supervisor calls the sub-agents as tools, in a fixed multi-step workflow.
trip_supervisor_agent = Agent(
    name="TripSupervisorAgent",
    model="gemini-2.5-flash",
    instruction="""
You are the Tahoe Ski Trip Planning Supervisor.

You MUST follow this workflow:

1. Call TripRequirementParserAgent → trip_request
2. Call TripSearchAgent → road_info
3. Call TripContextAgent using trip_request + road_info → trip_context
4. Call TripItineraryAgent using trip_request + trip_context + road_info → itinerary
5. Call TripSafetyAgent using trip_request + trip_context + itinerary + road_info → safety_notes

Finally respond to the user with:
- A short natural language summary
- The final itinerary (Markdown)
- A safety section (based on safety_notes)
- A brief reminder to check official road & weather sources before traveling.

Do NOT skip or reorder steps.
""",
    tools=[
        AgentTool(trip_requirement_parser_agent),
        AgentTool(trip_search_agent),
        AgentTool(trip_context_agent),
        AgentTool(trip_itinerary_agent),
        AgentTool(trip_safety_agent),
    ],
)


#### Observability

**Manual Test With Observability**

Below we run Sierra Summit AI using run_debug(), which surfaces:
* Each agent call
* Tool invocations
* Input/output JSON
* Supervisor summary
* Final answer

This demonstrates end-to-end agent interpretability.

In [18]:
## Runner & Single-Query Debug Test

runner = InMemoryRunner(agent=trip_supervisor_agent)

async def run_tahoe_trip_debug(user_query: str):
    logger.info("Starting Tahoe trip run", extra={"user_query": user_query})
    steps = await runner.run_debug(user_query)

    logger.info("Run completed", extra={"num_events": len(steps)})

    # Observability: print a compact trace
    print("\n=== Debug events (high-level) ===")
    for ev in steps:
        print(f"- author={ev.author}, role={ev.content.role}, finish={ev.finish_reason}")

    # Extract final supervisor answer
    final_event = steps[-1]
    final_text_parts = [
        p.text for p in final_event.content.parts
        if hasattr(p, "text") and p.text
    ]

    final_answer = final_text_parts[0] if final_text_parts else str(final_event.content)

    print("\n=== Final Answer ===\n")
    print(final_answer)

    logger.info("Final answer produced")
    return final_answer, steps


# Example test
test_query = "Plan a cheap 2-day ski trip to Tahoe next weekend for a beginner from San Jose. I want great views."

print("User >", test_query)
final_answer, steps = await run_tahoe_trip_debug(test_query)


2025-11-29 21:11:27,161 | INFO | Starting Tahoe trip run
INFO:tahoe_agent:Starting Tahoe trip run


User > Plan a cheap 2-day ski trip to Tahoe next weekend for a beginner from San Jose. I want great views.

 ### Created new session: debug_session_id

User > Plan a cheap 2-day ski trip to Tahoe next weekend for a beginner from San Jose. I want great views.


2025-11-29 21:11:48,607 | INFO | resort_lookup_tool called
INFO:tahoe_agent:resort_lookup_tool called
2025-11-29 21:11:48,609 | INFO | lodging_lookup_tool called
INFO:tahoe_agent:lodging_lookup_tool called
2025-11-29 21:11:48,610 | INFO | weather_tool called
INFO:tahoe_agent:weather_tool called
2025-11-29 21:12:34,200 | INFO | Run completed
INFO:tahoe_agent:Run completed
2025-11-29 21:12:34,201 | INFO | Final answer produced
INFO:tahoe_agent:Final answer produced


TripSupervisorAgent > Here is a plan for your cheap 2-day beginner ski trip to Lake Tahoe next weekend, departing from San Jose! You'll be staying in South Lake Tahoe and enjoying the scenic, beginner-friendly slopes of Northstar California. Expect mostly sunny to partly cloudy conditions, but be prepared for cold temperatures and potential light snow.

### Your Beginner Ski Trip to Lake Tahoe!

*   **Road Tip:** While no specific chain controls are currently in effect for the upcoming weekend, conditions can change quickly in mountainous areas. Always check Caltrans QuickMap or call 1-800-427-ROAD for real-time updates!

**Day 1: Travel & First Runs**
*   **Morning:** Depart San Jose for South Lake Tahoe. Check into your low-budget friendly Budget Inn South Lake, then make the scenic drive north to Northstar California.
*   **Afternoon:** Hit the beginner-friendly slopes at Northstar! Enjoy learning in a beautiful, scenic setting. Light snow is expected, making for classic cold but sk

#### Evaluation

This section implements a small, qualitative evaluation of Sierra Summit AI.

* Define a handful of evaluation scenarios
* Run each through run_debug()
* Extract lightweight metrics
* Present results in a table
* Score them using a simple rubric

The goal is to confirm consistent performance—not to create a large numerical benchmark.

In [19]:

# ==================================
# Evaluation Scenarios
# ==================================

EVAL_QUERIES = [
    {
        "id": "reasonable_intermediate_less_crowded",
        "query": "I’m an intermediate skier planning a day trip to Tahoe from Santa Clara. Keep the budget reasonable and recommend something less crowded.",
        "expected_skill": "intermediate",
        "expected_budget": "mid",
    },
    {
        "id": "advanced_south",
        "query": "I'm an advanced snowboarder coming from San Jose. Plan a 1-day trip to Tahoe with challenging terrain.",
        "expected_skill": "advanced",
    },
    {
        "id": "family_friendly",
        "query": "We are a family with kids, total beginners. 2 days in Tahoe with easy slopes and a cozy place to stay.",
        "expected_skill": "beginner",
    },
]


In [21]:
# ==================================
# Run Evaluation and Capture Results
# ==================================
#This runs each scenario, captures the final answer, and extracts lightweight metrics.
    
import pandas as pd

async def evaluate_agent():
    rows = []

    for case in EVAL_QUERIES:
        print(f"\n=== Running {case['id']} ===")
        
        final_answer, steps = await run_tahoe_trip_debug(case["query"])

        # Basic qualitative signals
        has_safety = "safety" in final_answer.lower() or "risk" in final_answer.lower()
        has_itinerary = "morning" in final_answer.lower() or "day" in final_answer.lower()
        mentions_skill = case["expected_skill"] in final_answer.lower()
        
        row = {
            "id": case["id"],
            "expected_skill": case.get("expected_skill"),
            "len_answer": len(final_answer),
            "has_safety": has_safety,
            "has_itinerary": has_itinerary,
            "mentions_skill": mentions_skill,
            "sample": final_answer[:200] + "..."
        }

        rows.append(row)

    return pd.DataFrame(rows)

eval_results = await evaluate_agent()
eval_results


2025-11-29 21:14:28,493 | INFO | Starting Tahoe trip run
INFO:tahoe_agent:Starting Tahoe trip run



=== Running reasonable_intermediate_less_crowded ===

 ### Continue session: debug_session_id

User > I’m an intermediate skier planning a day trip to Tahoe from Santa Clara. Keep the budget reasonable and recommend something less crowded.


2025-11-29 21:15:16,955 | INFO | resort_lookup_tool called
INFO:tahoe_agent:resort_lookup_tool called
2025-11-29 21:15:18,260 | INFO | lodging_lookup_tool called
INFO:tahoe_agent:lodging_lookup_tool called
2025-11-29 21:15:18,262 | INFO | weather_tool called
INFO:tahoe_agent:weather_tool called
2025-11-29 21:16:05,758 | INFO | Run completed
INFO:tahoe_agent:Run completed
2025-11-29 21:16:05,760 | INFO | Final answer produced
INFO:tahoe_agent:Final answer produced
2025-11-29 21:16:05,761 | INFO | Starting Tahoe trip run
INFO:tahoe_agent:Starting Tahoe trip run


TripSupervisorAgent > Here's a plan for your day ski trip to Tahoe from Santa Clara, tailored for an intermediate skier with a reasonable budget and a preference for fewer crowds.

While Northstar California can be popular, it offers excellent intermediate terrain and beautiful views. For a potentially less crowded experience, consider aiming for non-peak times or specific areas of the mountain.

### Northstar California Day Ski Trip Itinerary

**Note:** As of the general winter conditions provided (November 29, 2025), there are no chain controls in effect for major roadways into Lake Tahoe. However, conditions can change rapidly.

*   **Early Morning:** Depart from Santa Clara. Allow ample travel time to reach Northstar California in North Lake Tahoe.
*   **Late Morning:** Arrive at Northstar California. Head to the slopes and make the most of your day. Explore the mountain's intermediate runs, enjoying the varied terrain and scenic vistas.
*   **Afternoon:** Grab a quick and reasonab

2025-11-29 21:16:35,014 | INFO | resort_lookup_tool called
INFO:tahoe_agent:resort_lookup_tool called
2025-11-29 21:16:36,190 | INFO | lodging_lookup_tool called
INFO:tahoe_agent:lodging_lookup_tool called
2025-11-29 21:16:36,192 | INFO | weather_tool called
INFO:tahoe_agent:weather_tool called
2025-11-29 21:17:21,493 | INFO | Run completed
INFO:tahoe_agent:Run completed
2025-11-29 21:17:21,494 | INFO | Final answer produced
INFO:tahoe_agent:Final answer produced
2025-11-29 21:17:21,496 | INFO | Starting Tahoe trip run
INFO:tahoe_agent:Starting Tahoe trip run


TripSupervisorAgent > Here's a plan for your challenging 1-day snowboarding trip to Kirkwood from San Jose!

You're in for a treat with Kirkwood's renowned steep and advanced terrain. Expect colder temperatures and a chance of light snow, which should provide fresh, cold, and skiable conditions.

### Kirkwood Advanced Day Trip from San Jose

*   **Morning:** Hit the road early from San Jose towards South Lake Tahoe, heading straight for **Kirkwood Mountain Resort**. The drive should be smooth as **currently, no Caltrans chain controls are active for the main routes into Lake Tahoe, including US-50 and I-80.** Expect colder temperatures and a chance of light snow as you arrive.
*   **Late Morning/Afternoon:** Dive into Kirkwood's renowned steep and challenging terrain. Known for serious riders, you'll find plenty of advanced runs to keep you engaged. Light snow is expected, making for fresh, cold, but very skiable conditions!
*   **Late Afternoon:** Enjoy a final few runs as the day win

2025-11-29 21:17:52,885 | INFO | resort_lookup_tool called
INFO:tahoe_agent:resort_lookup_tool called
2025-11-29 21:17:54,389 | INFO | lodging_lookup_tool called
INFO:tahoe_agent:lodging_lookup_tool called
2025-11-29 21:17:54,391 | INFO | weather_tool called
INFO:tahoe_agent:weather_tool called
2025-11-29 21:18:39,352 | INFO | Run completed
INFO:tahoe_agent:Run completed
2025-11-29 21:18:39,353 | INFO | Final answer produced
INFO:tahoe_agent:Final answer produced


TripSupervisorAgent > Here's a plan for your cozy 2-day family ski trip to North Lake Tahoe, perfect for beginners with kids! You'll be staying in a comfortable condo at Northstar Village and enjoying the gentle slopes of Northstar California. The forecast for next weekend (December 6-7, 2025) is mostly sunny with clear skies, with a chance of light snow earlier in the week, making for cold but skiable conditions.

### Your Family-Friendly Northstar Ski Getaway

**Resort:** Northstar California (North Lake Tahoe)
**Lodging:** Northstar Village Condo
**Weather:** Light snow expected, cold but skiable – perfect for a cozy ski trip!

---

*   **Day 1: Arrival & First Slopes**
    *   **Morning:** Arrive in North Lake Tahoe. Check into your cozy Northstar Village Condo. Head to Northstar California to pick up rental gear and enroll the kids (or the whole family!) in a beginner ski lesson.
    *   **Afternoon:** Enjoy your first turns on Northstar's wide, mellow beginner slopes, known for b

,id,expected_skill,len_answer,has_safety,has_itinerary,mentions_skill,sample
0,reasonable_intermediate_less_crowded,intermediate,2918,True,True,True,Here's a plan for your day ski trip to Tahoe f...
1,advanced_south,advanced,3779,True,True,True,Here's a plan for your challenging 1-day snowb...
2,family_friendly,beginner,3416,True,True,True,Here's a plan for your cozy 2-day family ski t...


#### Scoring Rubric

Each evaluation scenario is assessed using a simple rule-based rubric:

Criterion	        Description	                                                               Points
* Safety Awareness	 Includes risks + mitigations	                                            1
* Itinerary Quality	 Provides a structured day-based itinerary	                                1
* Skill Alignment	 Mentions or reflects the expected skill (beginner/intermediate/advanced)	1
* Completeness	     Answer length > 400 characters (proxy for a full plan)	                    1
* Tool Pipeline	     Award full credit for completing the multi-agent pipeline	                1

Maximum Score = 5

This scoring method is simple, and transparent for qualitative agent evaluation.

In [22]:
# ==================================
# Scoring Function
# ==================================
def score_row(row):
    score = 0
    
    if row["has_safety"]:
        score += 1
    if row["has_itinerary"]:
        score += 1
    if row["mentions_skill"]:
        score += 1
    if row["len_answer"] > 400:
        score += 1
    
    # Award full credit for multi-agent pipeline (all scenarios)
    score += 1
    
    return score


# Apply scoring
eval_results["score"] = eval_results.apply(score_row, axis=1)
eval_results


,id,expected_skill,len_answer,has_safety,has_itinerary,mentions_skill,sample,score
0,reasonable_intermediate_less_crowded,intermediate,2918,True,True,True,Here's a plan for your day ski trip to Tahoe f...,5
1,advanced_south,advanced,3779,True,True,True,Here's a plan for your challenging 1-day snowb...,5
2,family_friendly,beginner,3416,True,True,True,Here's a plan for your cozy 2-day family ski t...,5


### Architecture Diagram

**Sequential Multi-Agent Orchestration**
- User → Supervisor
- Supervisor → calls Parser
- Supervisor → calls Search
- Supervisor → calls Context
- Supervisor → calls Itinerary
- Supervisor → calls Safety
- Supervisor → composes final answer

In [ ]:
[ User Query ]
        |
        v
+--------------------------+
|  TripSupervisorAgent     |
|  (Orchestration Engine)  |
+--------------------------+
        |
        v
+------------------------------+
|  TripRequirementParserAgent  |
|  -> trip_request             |
+------------------------------+
        |
        v
+------------------------------+
|     TripSearchAgent          |
|   (calls google_search)      |
|  -> road_info                |
+------------------------------+
        |
        v
+------------------------------------------+
|            TripContextAgent               |
|   (resort_lookup, lodging_lookup,         |
|      weather_tool -> trip_context)        |
+------------------------------------------+
        |
        v
+------------------------------+
|     TripItineraryAgent       |
|    -> itinerary (Markdown)   |
+------------------------------+
        |
        v
+------------------------------+
|      TripSafetyAgent         |
|       -> safety_notes        |
+------------------------------+
        |
        v
+------------------------------------------------+
|        Final Answer (Supervisor Output)        |
| - Natural Language Summary                      |
| - Markdown Itinerary                            |
| - Safety Notes                                   |
+------------------------------------------------+

        ^
        |
+-------------------------------+
| Observability & Evaluation    |
| - Logging (tools, steps)      |
| - Debug runner                |
| - Eval queries                |
+-------------------------------+
